In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import *
import os
import sys 

In [2]:
os.environ["JAVA_HOME"] = "C:\Program Files\Java\jdk-18.0.2.1"

In [3]:
conf = SparkConf() \
    .setAppName("ETLPipeline") \
    .setMaster("local") \
    .set("spark.driver.extraClassPath","C:/pyspark/*")

In [4]:
sc = SparkContext.getOrCreate(conf=conf)
etl = SparkSession(sc)

In [12]:
username_src = "spark_sql"
username_target = "postgres"
pass_src = "Saifullah2398"
pass_target = "Saifullah2398"
server_src = "DESKTOP-PJRE6QD\SQLEXPRESS"
server_target = "localhost"
src_db = "dimasdb"
target_db = "postgres"
src_driver = "com.microsoft.sqlserver.jdbc.SQLServerDriver"
target_driver = "org.postgresql.Driver"
src_url = f"jdbc:sqlserver://{server_src}:1433;databaseName={src_db};user={username_src};password={pass_src};encrypt=true;trustServerCertificate=true"
target_url = f"jdbc:postgresql://{server_target}:5432/{target_db}?user={username_target}&password={pass_target};encrypt=true;trustServerCertificate=true"

In [13]:
sql = """ SELECT t.name as table_name FROM sys.tables t
        WHERE t.name IN ( 'contract_loan','contract_loan_ledger' )"""

In [14]:
dfs = etl.read. \
      format("jdbc"). \
      options(driver=src_driver,user=username_src,url=src_url,query=sql). \
      load()
dfs.show()

+--------------------+
|          table_name|
+--------------------+
|       contract_loan|
|contract_loan_ledger|
+--------------------+



In [15]:
data_collect = dfs.collect()
for row in data_collect:
    print(row['table_name'])

contract_loan
contract_loan_ledger


In [16]:
def load(df, tbl):
    try:
        rows_imported = 0
        print(f"importing rows {rows_imported} to {rows_imported + df.count()} for table {tbl}")
        df.write.mode('overwrite') \
        .format('jdbc') \
        .option('url', target_url) \
        .option('user', username_target) \
        .option('password', pass_target) \
        .option('driver', target_driver) \
        .option('dbtable', 'src_' + tbl) \
        .save()
        print('Data imported Successful')
        rows_imported += df.count()
    except Exception as e:
        print("Data load error : " + str(e))

In [17]:
def extract():
    try:
        dfs = etl.read. \
            format('jdbc'). \
            options(driver=src_driver,user=username_src,url=src_url,query=sql). \
            load()
        data_collect = dfs.collect()
        for row in data_collect:
            print(row['table_name'])
            tbl_name = row['table_name']
            df = etl.read \
            .format("jdbc") \
            .option('driver', src_driver) \
            .option('user', username_src) \
            .option('password', pass_src) \
            .option('url', src_url) \
            .option('dbtable', f"dbo.{tbl_name}") \
            .load()
            load(df,tbl_name)
            print('Data loaded Succesfully')
    except Exception as e:
        print('Data extract error : ' + str(e))

In [18]:
extract()

contract_loan
importing rows 0 to 100 for table contract_loan
Data imported Successful
Data loaded Succesfully
contract_loan_ledger
importing rows 0 to 1097 for table contract_loan_ledger
Data imported Successful
Data loaded Succesfully
